In [ ]:
# import torch
# import IPython
# from IPython.display import Image
# import tensorboard
# import tensorflow
# import speech_recognition as sr
# import pocketsphinx
# import pyaudio
# recognizer = sr.Recognizer()
# import os
# import cv2


In [1]:
def is_object_cutoff(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox
    if x_min <= 0 or x_max >= img_width or y_min <= 0 or y_max >= img_height:
        return True  # Object is cut off
    return False

In [2]:
def Detect(frame, target_class):
    # הרצת המודל על התמונה
    results = model(frame)
    num_class= int(target_class)
    # המרת תוצאות ל-DataFrame
    df = results.pandas().xyxy[0]
    print(df)
    # סינון לפי המחלקה המבוקשת (class) ושמירה של התיבה עם ההסתברות הגבוהה ביותר
    filtered_df = df[df['class'] == num_class]
    
    # בדיקה אם אין תוצאות למחלקה המבוקשת
    if filtered_df.empty:
        print(f"לא נמצאו תוצאות עבור מחלקה {target_class}")
        return None

    # מציאת התיבה עם ההסתברות הגבוהה ביותר (העמודה 'confidence')
    best_row = filtered_df.loc[filtered_df['confidence'].idxmax()]

    # חישוב רוחב וגובה
    xmin, ymin, xmax, ymax = best_row['xmin'], best_row['ymin'], best_row['xmax'], best_row['ymax']
    bbox= xmin, ymin, xmax, ymax 
    width = xmax - xmin
    height = ymax - ymin
    is_object_cutoff= is_object_cutoff(bbox, width,height)
    
    print(f"התיבה התוחמת למחלקה {target_class} עם ההסתברות הגבוהה ביותר:")
    print(f"רוחב = {width}, גובה = {height}, הסתברות = {best_row['confidence']}")

    return height, is_object_cutoff